In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
transaction = pd.read_csv('./data/transaction_new.csv')
customer = pd.read_csv('./data/customer.csv')

In [3]:
customer = customer.assign(first_join_date=pd.to_datetime(customer['first_join_date'])+timedelta(hours=7))
transaction = transaction.assign(created_at=pd.to_datetime(transaction['created_at'])+timedelta(hours=7))
transaction = transaction.assign(created_at = transaction['created_at'].dt.strftime('%Y-%m-%d'))
transaction = transaction.assign(created_at = pd.to_datetime(transaction['created_at']))
transaction = transaction.assign(total_price=transaction['total_amount'] - transaction['shipment_fee'])

In [5]:
customer_date = customer[['customer_id', 'first_join_date']].sort_values('first_join_date').reset_index(drop=True)
customer_date

,customer_id,first_join_date
0,74089,2016-06-30 07:00:00
1,21734,2016-06-30 07:00:00
2,38119,2016-06-30 07:00:00
3,73773,2016-06-30 07:00:00
4,60006,2016-06-30 07:00:00
...,...,...
99995,73572,2022-07-31 07:00:00
99996,66776,2022-07-31 07:00:00
99997,96644,2022-07-31 07:00:00
99998,27995,2022-07-31 07:00:00


* ARPU
    * 매출(주 단위) / 주 단위로 누적(유입) 유저
* ARPPU
    * 매출(주 단위) / 주 단위로 누적 구매 유저

In [58]:
transaction_by_session = transaction.groupby('session_id')[['created_at', 'customer_id', 'total_price']].max().sort_values('created_at').reset_index(drop=True)
transaction_by_session

,created_at,customer_id,total_price
0,2016-07-01,6183,568826
1,2016-07-01,74089,630795
2,2016-07-01,16228,318792
3,2016-07-02,73773,246670
4,2016-07-02,2181,299345
...,...,...,...
852579,2022-08-01,88855,1375460
852580,2022-08-01,5733,198721
852581,2022-08-01,88356,261067
852582,2022-08-01,15753,142200


In [4]:
data_start_date = pd.to_datetime('2016-06-30')
previous_start_date = pd.to_datetime('2016-07-11')
previous_end_date = previous_start_date + timedelta(days=6)

current_start_date = previous_start_date + timedelta(days=7)
current_end_date = previous_start_date + timedelta(days=13)

print(previous_start_date, previous_end_date)
print(current_start_date, current_end_date)

2016-07-11 00:00:00 2016-07-17 00:00:00
2016-07-18 00:00:00 2016-07-24 00:00:00


In [7]:
prev_customer = customer_date[(customer_date.first_join_date>=data_start_date)&
                              (customer_date.first_join_date<=previous_end_date)]

curr_customer = customer_date[(customer_date.first_join_date>=data_start_date)&
                              (customer_date.first_join_date<=current_end_date)]

In [11]:
prev_transaction = transaction_by_session[(transaction_by_session.created_at>=previous_start_date)&
                                          (transaction_by_session.created_at<=previous_end_date)]

curr_transaction = transaction_by_session[(transaction_by_session.created_at>=current_start_date)&
                                          (transaction_by_session.created_at<=current_end_date)]

prev_transaction_cum = transaction_by_session[(transaction_by_session.created_at>=data_start_date)&
                                              (transaction_by_session.created_at<=previous_end_date)]

curr_transaction_cum = transaction_by_session[(transaction_by_session.created_at>=data_start_date)&
                                              (transaction_by_session.created_at<=current_end_date)]

In [12]:
# ARPU

print('prev', prev_transaction.total_price.sum() / prev_customer.customer_id.nunique())
print('curr', curr_transaction.total_price.sum() / curr_customer.customer_id.nunique())

prev 65924.27792207792
curr 63809.98093587522


In [13]:
# ARPPU
print('prev', prev_transaction.total_price.sum() / prev_transaction_cum.customer_id.nunique())
print('curr', curr_transaction.total_price.sum() / curr_transaction_cum.customer_id.nunique())

prev 172658.8231292517
curr 178729.89805825244


* ARPU
    * 매출(월 단위) / 월 단위로 누적(유입) 유저
* ARPPU
    * 매출(주 단위) / 월 단위로 누적 구매 유저

In [59]:
customer_date = customer_date.assign(first_join_month = customer_date['first_join_date'].dt.strftime('%Y-%m'))
transaction_by_session = transaction_by_session.assign(purchase_month = transaction_by_session['created_at'].dt.strftime('%Y-%m'))

display(customer_date)
display(transaction_by_session)

,customer_id,first_join_date,first_join_month
0,74089,2016-06-30 07:00:00,2016-06
1,21734,2016-06-30 07:00:00,2016-06
2,38119,2016-06-30 07:00:00,2016-06
3,73773,2016-06-30 07:00:00,2016-06
4,60006,2016-06-30 07:00:00,2016-06
...,...,...,...
99995,73572,2022-07-31 07:00:00,2022-07
99996,66776,2022-07-31 07:00:00,2022-07
99997,96644,2022-07-31 07:00:00,2022-07
99998,27995,2022-07-31 07:00:00,2022-07


,created_at,customer_id,total_price,purchase_month
0,2016-07-01,6183,568826,2016-07
1,2016-07-01,74089,630795,2016-07
2,2016-07-01,16228,318792,2016-07
3,2016-07-02,73773,246670,2016-07
4,2016-07-02,2181,299345,2016-07
...,...,...,...,...
852579,2022-08-01,88855,1375460,2022-08
852580,2022-08-01,5733,198721,2022-08
852581,2022-08-01,88356,261067,2022-08
852582,2022-08-01,15753,142200,2022-08


In [60]:
FOR_USER = customer_date[customer_date['first_join_month']<='2017-01']
FOR_PU = transaction_by_session[transaction_by_session['purchase_month']<='2017-01']
FOR_REVENUE = transaction_by_session[transaction_by_session['purchase_month'].str.contains('2017')]

In [61]:
FOR_USER

,customer_id,first_join_date,first_join_month
0,74089,2016-06-30 07:00:00,2016-06
1,21734,2016-06-30 07:00:00,2016-06
2,38119,2016-06-30 07:00:00,2016-06
3,73773,2016-06-30 07:00:00,2016-06
4,60006,2016-06-30 07:00:00,2016-06
...,...,...,...
5540,87033,2017-01-31 07:00:00,2017-01
5541,37491,2017-01-31 07:00:00,2017-01
5542,3127,2017-01-31 07:00:00,2017-01
5543,87539,2017-01-31 07:00:00,2017-01


In [62]:
USER = FOR_USER.customer_id.nunique()
USER

5545

In [63]:
FOR_PU

,created_at,customer_id,total_price,purchase_month
0,2016-07-01,6183,568826,2016-07
1,2016-07-01,74089,630795,2016-07
2,2016-07-01,16228,318792,2016-07
3,2016-07-02,73773,246670,2016-07
4,2016-07-02,2181,299345,2016-07
...,...,...,...,...
6344,2017-01-31,61726,440955,2017-01
6345,2017-01-31,65058,138992,2017-01
6346,2017-01-31,18829,168989,2017-01
6347,2017-01-31,74053,197093,2017-01


In [64]:
PU = FOR_PU.customer_id.nunique()
PU

2996

In [65]:
REVENUE = FOR_REVENUE.groupby('purchase_month')['total_price'].sum().to_frame().reset_index()
REVENUE

,purchase_month,total_price
0,2017-01,784823105
1,2017-02,854579476
2,2017-03,1078665421
3,2017-04,1118774573
4,2017-05,1312819630
5,2017-06,1253000789
6,2017-07,1806633910
7,2017-08,1741156129
8,2017-09,1902858220
9,2017-10,2134949407


In [53]:
# ARPPU

REVENUE[REVENUE['purchase_month']=='2017-01']['total_price'] / PU

0    261956.977637
Name: total_price, dtype: float64

In [68]:
784823105 / 2996

261956.97763684913

In [57]:
# ARPU

REVENUE[REVENUE['purchase_month']=='2017-01']['total_price'] / USER

0    141537.079351
Name: total_price, dtype: float64

In [67]:
784823105 / 5545

141537.07935076646